In [1]:
# !pip install transformers -U
!pip install git+https://github.com/huggingface/transformers

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-89h6sde1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-89h6sde1
  Resolved https://github.com/huggingface/transformers to commit 6ae71ec8369f71490311b30c902ea6990efda4f0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [5]:
!pip install -qU bitsandbytes transformers datasets accelerate loralib einops xformers
!pip install -q -U git+https://github.com/huggingface/peft.git

DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
import torch
from transformers import LlamaForCausalLM, CodeLlamaTokenizer, BitsAndBytesConfig

tokenizer = CodeLlamaTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")
model = LlamaForCausalLM.from_pretrained("codellama/CodeLlama-7b-hf",
    device_map='auto',
    load_in_4bit=True,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4'
		))

Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
model_orig = model

In [7]:
import os
import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from peft import (
    LoraConfig,
    PeftConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)

model = prepare_model_for_kbit_training(model)


In [10]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
# https://github.com/tloen/alpaca-lora for q_proj v_proj

# https://github.com/mzbac/qlora-fine-tune TODO: PEEP

## Setup Training Data
<!-- https://medium.com/@amodwrites/a-definitive-guide-to-qlora-fine-tuning-falcon-7b-with-peft-78f500a1f337 -->

In [12]:
from datasets import load_dataset

def generate_prompt(data_point):
  return f"""
<Human>: {data_point["Context"]}
<AI>: {data_point["Response"]}
  """.strip()

tokenizer.pad_token = tokenizer.eos_token
def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

dataset_name = 'Amod/mental_health_counseling_conversations'
dataset = load_dataset(dataset_name, split="train")

dataset = dataset.shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [14]:
# TODO: via fast.ai
# Nice
"""
from fastai.text import *

# Path to your data
path = Path('data/texts')

# Create a DataBunch suitable for language modeling
data_lm = (TextList.from_folder(path)
           .filter_by_folder(include=['.'])  # Include all files in the root directory of 'path'
           .split_by_rand_pct(0.1)  # Randomly split 10% of data for validation
           .label_for_lm()  # Use this method for a language model
           .databunch(bs=64, num_workers=1))

data_lm.save('data_lm.pkl')
"""
# From chatgpt
dataset[1]

{'Context': "I'm a female in my mid 20s.  Lately I tend to over drink and I've become a very angry drunk. \r\n\r\nIn the past, I have even cheated on my boyfriend while I was under the influence of alcohol. \r\n\r\nBut now, even if I don't do anything wrong and don't embarrass myself, I still feel really guilty after a night of drinking. I don't understand why I'm feeling this way. Does this mean I have a problem?",
 'Response': 'Hello, I commend you for your courage in taking a look at the role alcohol has in your life. It sounds like you\'re concerned about what happens when you drink too much and I suspect you already know the answer to your question about whether you have a problem or not. I imagine you would like to stop feeling guilty and would like to avoid cheating on your boyfriend or other negative consequences and maybe have a fear of being or becoming an "addict" or "alcoholic." You might have a "problem" but that does not necessarily mean that you are an addict.\xa0I don\'

## Setup training

In [15]:
OUTPUT_DIR = 'out'

training_args = transformers.TrainingArguments(
    auto_find_batch_size=True,
    num_train_epochs=4,
    learning_rate=2e-4,
    bf16=True,
    save_total_limit=4,
    logging_steps=10,
    output_dir=OUTPUT_DIR,
    save_strategy='epoch',
)

## Train

In [16]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train()

2023-09-27 15:09:10.185371: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-27 15:09:10.200525: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-27 15:09:10.645403: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


ValueError: You cannot perform fine-tuning on purely quantized models. Please attach trainable adapters on top of the quantized model to correctly perform fine-tuning. Please see: https://huggingface.co/docs/transformers/peft for more details